# SITCOM-1399: Adding Earthquake events to the TN81, on vibration on M1M3

**Descripción**

Sub-ticket related to the [SITCOM-918: Write technote on hardpoint oscillations during tma slews](https://rubinobs.atlassian.net/browse/SITCOM-918), to study earthquake events seen by M1M3. 
We can read the TN81 [here](https://sitcomtn-081.lsst.io/v/SITCOM-918/index.html).

See more about the ticket in: [https://rubinobs.atlassian.net/browse/SITCOM-1399](https://rubinobs.atlassian.net/browse/SITCOM-1399)



**More information**

The **VMS (Vibration Monitoring System)** is a set of accelerometers mounted on the M1M3, M2, and Camera Rotator that measure high frequency accelerations (not including gravity). The data rate is 200 hz, and so can be used to identify vibrations up to 100 hz. More information can be found [confluence](https://confluence.lsstcorp.org/x/jQhUCQ). There are multiple sensors on each of the three components. In particular, for M1M3 has 3 sensors.

The VMS data for the mag~6 earthquake of September are shown in slides no. 56 and 57 available [here](https://docs.google.com/presentation/d/1HmmzIUt0XszK0XMS1YZtQiYCvdwajhrZ8p3ZdAVSp14/edit#slide=id.g2633a5ccbdd_1_394).

In this notebook we will study whether this type of event is safe for the mirror.

The dates of large earthquakes are:
- 2023-09-06 23:48:15 UTC
- 2023-10-31 12:33:43 UTC

## Function the read VMS data

In [1]:
import sys, time, os, asyncio
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay
%matplotlib inline
%load_ext autoreload
%autoreload 2